In [1]:
import sys
import os
import pickle
import pandas as pd
import json
import datetime
import time
import copy
import numpy as np

py_file_location = '../PrivatePackages'
sys.path.append(os.path.abspath(py_file_location))
from sklearn.model_selection import train_test_split

import JiaoCheng
import NingXiang

/Applications/anaconda3/lib/python3.9/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.5
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [2]:
LABEL = 'trend'
MODEL = 'lstmc'

In [3]:
curated_data = pd.read_csv('../data/curated_data.csv')
curated_data = curated_data[['date', 
                        'stock',
                        'trend',
                        'closePrice_lag_1', 
                        'log_ret_normalised_by_day_lag_1', 
                        'closePriceNorm_lag_1', 
                        'log_ret_lag_1', 
                        'mean_log_ret_lag_1',
                        'mean_closePrice_lag_1',
                        'mean_closePriceNorm_lag_1',
                        'mean_log_ret_normalised_by_day_lag_1', 
                        'pos_log_ret_lag_1',
                        'pos_closePrice_lag_1',
                        'pos_closePriceNorm_lag_1',
                        'pos_log_ret_normalised_by_day_lag_1']]
curated_data['date'] = pd.to_datetime(curated_data['date'])

In [4]:
date = [datetime.datetime(year=2022, month=12, day = 31) + datetime.timedelta(days=i) for i in range(250)]
val_date_start = date[175]
test_date_start = date[175+38]

In [5]:
curated_data['date'] = pd.to_datetime(curated_data['date'])
curated_data = curated_data.dropna()

train = curated_data[(curated_data['date'] < val_date_start)]
val = curated_data[(curated_data['date'] < test_date_start) & (curated_data['date'] >= val_date_start)]
test = curated_data[(curated_data['date'] >= test_date_start)]

In [6]:
def data_factory(data, lag):
    x_list, y_list = list(), list()

    for id, stock_data in data.groupby('stock'):
        stock_data.sort_values(by='date', inplace=True)
        stock_data.drop(['stock', 'date'], axis=1, inplace=True)
        stock_data.index = range(len(stock_data))

        for i in range(len(stock_data)):
            
            if i < lag-1:
                continue

            x = stock_data.iloc[i-lag+1:i+1].values.copy()
            x[-1:, 0] = 0 # mask last day's return
            y = stock_data.loc[i]['trend']
            
            x_list.append(x)
            y_list.append(y)
    
    return x_list, y_list

In [7]:
LAG = 5

train_x, train_y = data_factory(train, LAG)
val_x, val_y = data_factory(val, LAG)
test_x, test_y = data_factory(test, LAG)

In [8]:
jiaocheng = JiaoCheng.JiaoCheng()

from models.LSTM import LSTMC_pt as LSTMC

jiaocheng.read_in_model(LSTMC, 'Classification')

jiaocheng.read_in_data(train_x, train_y, val_x, val_y, test_x, test_y)

parameter_choices = {
    'lstm_hidden_layer_n_neurons': [6, 12, 25, 50, 100],
    'lstm_n_hidden_layers': [1, 2, 3],
    'bi_lstm': [False, True],
    'n_hidden_layers': [1, 2, 3],
    'batch_size': [64, 128, 256, 512],
    'learning_rate': [0.001, 0.01, 0.1, 1],
    'dense_hidden_layer_n_neurons': [6, 12, 25, 50, 100],
    'activation': ['relu', 'sigmoid', 'tanh', 'softmax'],
    'dropout_prob': [0, 0.1, 0.3, 0.5],
    'batch_normalisation': [False, True],
    'dense_layer_type': ['Dense', 'Residual'],
    'attention_num_heads': [0, 1],

}

jiaocheng.set_hyperparameters(parameter_choices)

jiaocheng.set_non_tuneable_hyperparameters({'verbose' : True, 'random_state' : 19260817, 'loss_function':'CrossEntropy', 'num_epochs':250, 'grad_clip': False})

jiaocheng.set_tuning_order(['lstm_n_hidden_layers', 'n_hidden_layers', 'lstm_hidden_layer_n_neurons', 'dense_hidden_layer_n_neurons', 
                            'bi_lstm', 'attention_num_heads', 'activation', 'dense_layer_type',
                            'dropout_prob', 'batch_size', 'batch_normalisation', 'learning_rate'])

jiaocheng.set_hyperparameter_default_values({
    'lstm_hidden_layer_n_neurons': 6,
    'lstm_n_hidden_layers': 1,
    'bi_lstm': False,
    'n_hidden_layers': 1,
    'batch_size': 128,
    'learning_rate': 0.01,
    'dense_hidden_layer_n_neurons': 6,
    'activation': 'relu',
    'dropout_prob': 0.1,
    'batch_normalisation': False,
    'dense_layer_type': 'Dense',
    'attention_num_heads': 0}
)

try:
    jiaocheng.read_in_tuning_result_df(f'../models/tuning/jiaocheng_{MODEL}_{LABEL}.csv')
except:
    pass

jiaocheng.set_tuning_result_saving_address(f'../models/tuning/jiaocheng_{MODEL}_{LABEL}.csv')
jiaocheng.set_best_model_saving_address(f'../models/tmp_models/jiaocheng_{MODEL}_{LABEL}.pickle')

jiaocheng.tune()

print('\n\n')

JiaoCheng Initialised
Successfully read in model <class 'models.LSTM.LSTMC_pt'>, which is a Classification model
Read in Train X data
Read in Train y data
Read in Val X data
Read in Val y data
Read in Test X data
Read in Test y data
Successfully recorded hyperparameter choices
Successfully recorded non_tuneable_hyperparameter choices
Error message: 'dict' object is not callable
Error Importing this Row: (0, lstm_hidden_layer_n_neurons              6.0
lstm_n_hidden_layers                     1.0
bi_lstm                                  0.0
n_hidden_layers                          1.0
batch_size                             128.0
learning_rate                           0.01
dense_hidden_layer_n_neurons             6.0
activation                              relu
dropout_prob                             0.1
batch_normalisation                      0.0
dense_layer_type                       Dense
attention_num_heads                      0.0
Train accu                          0.502249
Val 

/Applications/anaconda3/lib/python3.9/site-packages/torch/nn/modules/rnn.py:67: UserWarning: dropout option adds dropout after all but last recurrent layer, so non-zero dropout expects num_layers greater than 1, but got dropout=0.1 and num_layers=1
  warnings.warn("dropout option adds dropout after all but last "
/Users/tg.chenny/Desktop/3. Self Projects/2. PythonProject/Incomplete/DeepTrade/PrivatePackages/models/LSTM.py:15: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at /Users/runner/work/pytorch/pytorch/pytorch/torch/csrc/utils/tensor_new.cpp:233.)
  self.x = torch.FloatTensor(x)
/Users/tg.chenny/Desktop/3. Self Projects/2. PythonProject/Incomplete/DeepTrade/PrivatePackages/models/LSTM.py:221: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  s

Epoch [100/250], Loss: 0.6932
Epoch [200/250], Loss: 0.6932


/Applications/anaconda3/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Applications/anaconda3/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Applications/anaconda3/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Trained and Tested combination 15 of 921600: (0, 0, 0, 0, 1, 1, 0, 0, 1, 0, 0, 0), taking 241.37 seconds to get val score of 0.5024
        Current best combo: (0, 0, 0, 0, 1, 1, 0, 0, 1, 0, 0, 0) with val score 0.5024


/Users/tg.chenny/Desktop/3. Self Projects/2. PythonProject/Incomplete/DeepTrade/PrivatePackages/models/LSTM.py:221: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  softmax_out = self.softmax(out)


Epoch [100/250], Loss: 0.6932


RuntimeError: each element in list of batch should be of equal size

In [21]:
class CustomDataLoader(Dataset):
    def __init__(self, x, y = None):
        self.x = torch.FloatTensor(x)
        if y is not None:
            if type(y) == np.ndarray:
                self.y = torch.FloatTensor(y)
            elif type(y) == pd.core.series.Series or type(y) == pd.core.frame.DataFrame:
                self.y = y
        self.len = len(x)
        
    def __len__(self):
        return self.len
    
    def __getitem__(self, idx):
        try:
            if type(self.y) == pd.core.series.Series:
                y = torch.tensor(self.y.iloc[idx], dtype=torch.float32)
                return self.x[idx], y
            elif type(self.y) == pd.core.frame.DataFrame:
                y = torch.tensor(self.y.iloc[idx].values, dtype=torch.float32)
                return self.x[idx], y
            else:
                self.x[idx], self.y[idx]
        except:
            return self.x[idx]

In [26]:
class CustomDataLoader(Dataset):
    def __init__(self, x, y = None):
        self.x = torch.FloatTensor(x)
        if y is not None:
            if type(y) == np.ndarray or type(y) == list:
                self.y = torch.FloatTensor(y)
            elif type(y) == pd.core.series.Series or type(y) == pd.core.frame.DataFrame:
                self.y = y
        self.len = len(x)

        print(self.x)
        print(self.y)
        
    def __len__(self):
        return self.len
    
    def __getitem__(self, idx):
        try:
            if type(self.y) == pd.core.series.Series:
                y = torch.tensor(self.y.iloc[idx], dtype=torch.float32)
                return self.x[idx], y
            elif type(self.y) == pd.core.frame.DataFrame:
                y = torch.tensor(self.y.iloc[idx].values, dtype=torch.float32)
                return self.x[idx], y
            else:
                self.x[idx], self.y[idx]
        except:
            return self.x[idx]

In [27]:
train_dataset = CustomDataLoader(train_x, train_y)
train_loader = DataLoader(train_dataset, batch_size=128, shuffle=True)

In [28]:
train_y

[1.0,
 -1.0,
 1.0,
 1.0,
 -1.0,
 -1.0,
 -1.0,
 1.0,
 -1.0,
 -1.0,
 1.0,
 1.0,
 -1.0,
 1.0,
 1.0,
 -1.0,
 -1.0,
 1.0,
 -1.0,
 -1.0,
 1.0,
 1.0,
 -1.0,
 -1.0,
 1.0,
 1.0,
 1.0,
 -1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 -1.0,
 -1.0,
 1.0,
 1.0,
 -1.0,
 -1.0,
 -1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 -1.0,
 1.0,
 1.0,
 1.0,
 -1.0,
 1.0,
 1.0,
 1.0,
 -1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 -1.0,
 -1.0,
 -1.0,
 -1.0,
 1.0,
 -1.0,
 1.0,
 -1.0,
 1.0,
 1.0,
 -1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 -1.0,
 -1.0,
 -1.0,
 1.0,
 1.0,
 -1.0,
 1.0,
 1.0,
 -1.0,
 -1.0,
 1.0,
 1.0,
 1.0,
 -1.0,
 -1.0,
 1.0,
 1.0,
 -1.0,
 1.0,
 -1.0,
 1.0,
 1.0,
 -1.0,
 -1.0,
 -1.0,
 -1.0,
 1.0,
 -1.0,
 -1.0,
 -1.0,
 1.0,
 1.0,
 1.0,
 -1.0,
 1.0,
 -1.0,
 1.0,
 -1.0,
 -1.0,
 -1.0,
 1.0,
 1.0,
 -1.0,
 -1.0,
 -1.0,
 1.0,
 -1.0,
 -1.0,
 -1.0,
 -1.0,
 -1.0,
 -1.0,
 -1.0,
 -1.0,
 -1.0,
 1.0,
 -1.0,
 1.0,
 -1.0,
 -1.0,
 1.0,
 -1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 -1.0,
 -1.0,
 1.0,
 1.0,
 -1.0,
 -1.0,
 -1.0,
 -1.0,
 -1.0,
 -1.0,
 -1.0,
 -1.0,
 -1.0

In [29]:
for batch_idx, (x, y) in enumerate(train_loader):
    print(x)


TypeError: default_collate: batch must contain tensors, numpy arrays, numbers, dicts or lists; found <class 'NoneType'>